# MCap Performance Index

In [3]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore') #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; 
    else set it to False to turn off debugging mode '''
debug = True

In [4]:
import os
import sys
from datetime import datetime, date, timedelta

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
import rezaware as reza
from mining.modules.assets.etp import performIndex as indx
from utils.modules.ml.timeseries import rollingstats as stats
from utils.modules.etl.load import sparkNoSQLwls as nosql
from utils.modules.lib.spark import execSession as spark

''' restart initiate classes '''
if debug:
    import importlib
    reza = importlib.reload(reza)
    indx = importlib.reload(indx)
    stats= importlib.reload(stats)
    nosql= importlib.reload(nosql)
    spark= importlib.reload(spark)
    
__desc__ = "analyze crypto market capitalization time series data"
clsIndx =indx.Portfolio(desc=__desc__)
clsStat=stats.RollingStats(desc=__desc__)
clsNoSQL=nosql.NoSQLWorkLoads(desc=__desc__)
print("\nClass initialization and load complete!")

All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional PERFORMINDEX-libraries in ETP-package of ASSETS-module imported successfully!
All packages in utils ml timeseries RollingStats imported successfully!
All functional SPARKNOSQLWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional EXECSESSION-libraries in SPARK-package of LIB-module imported successfully!
All functional SPARKCLEANNRICH-libraries in TRANSFORM-package of ETL-module imported successfully!
sparkNoSQLwls Class initialization complete
execSession Class initialization complete
All functional DAILYTOPN-libraries in ETP-package of ASSETS-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
sparkNoSQLwls Class initialization complete
dailyTopN Class initialization complete
performIndex Class initialization complete
execSession Class initialization complete
sparkNoSQLwls Class initiali

## Read Portfolio

In [5]:
_db_coll=["mpt.top3.2022-01-10","mpt.top3.2022-01-09"]
_coll_dt=date(2022,1,10)
_mpt_port = clsIndx.get_portfolio(
    db_name="tip-daily-mpt",
    db_coll=[],#_db_coll,
    coll_date=_coll_dt,
)
_mpt_port

[{'_id': ObjectId('63d4bb80badb4157b3025cb3'),
  'date': '2022-01-10T00:00:00',
  'asset': 'charg_coin',
  'mcap.weight': 0.4453207926976063,
  'mcap.ror': 0.6574680456259401,
  'mcap.value': 1148729.3496556643},
 {'_id': ObjectId('63d4bb80badb4157b3025cb4'),
  'date': '2022-01-10T00:00:00',
  'asset': 'matryx',
  'mcap.weight': 0.34821271988748376,
  'mcap.ror': 0.2861116369010913,
  'mcap.value': 2246173.234597205},
 {'_id': ObjectId('63d4bb80badb4157b3025cb5'),
  'date': '2022-01-10T00:00:00',
  'asset': 'bitcoinpos',
  'mcap.weight': 0.20646648741490986,
  'mcap.ror': 0.20342319564693787,
  'mcap.value': 14522857.416871611}]

## Compute Index Values
Index computations
* RSI - Relative Strength Index
* DMI - Directional Movement Indicator
* ADX - Average Directional Index

### RSI

In [ ]:
''' TBD '''

### DMI
* The [calculation is somewhat complex](https://www.investopedia.com/ask/answers/112814/what-directional-movement-index-dmi-formula-and-how-it-calculated.asp); defined in the class ```mining/modules/assets/etp/performIndex```.
* [DMI indicates how strongly a price is moving](https://www.investopedia.com/articles/technical/02/050602.asp); ranges from a low of 0 to a high of 100.
   * DMI+ trending upwards; 
      * DMI > 25 : price is directionally strong.
      * Bullish - peak indicates a swing high.
   * DMI- trending downwards;
      * DMI < 25 : price is directionally weak.
      * Bearish - peak low indicates a swing low.
   * DMI+ > 25 & DMI- < 25 : buyers are stronger than sellers.
   * DMI+ < 25 & DMI- > 25 : sellers are stronger than the buyers.
* Strong uptrend will show a series of rising +DMI peaks that remain above the -DMI for extended periods of time
* A [crossover](https://www.investopedia.com/terms/c/crossover.asp) indicates when to go long or short but __advised to look at other indicators__ because they frequently give false signals when volatility is low and late signals when volatility is high.
* Use crossovers as the __first indication__ of a potential change in direction.
* When price makes a new pivot high and the +DMI makes a new high is one method of __confirming an asset's uptrend__.

In [6]:
''' Get DMI data '''
__idx_type__='dmi'
_kwargs={
    "LOGCOLNAME" :'log_ror',
    "PARTCOLNAME":"asset_name",
    "VALCOLNAME" :"mcap_value",
    "PREVALCOLNAME":'mcap_prev_val',
    "DIFFCOLNAME":'mcap_diff',
    "WINLENGTH":7,
    "WINUNIT":'DAY',
}
_idx = clsIndx.get_dmi_data(
    dmi_sdf_= 
        val_col:str="log_ror",
        part_col:str="asset_name",
    **_kwargs,
)

print("Retrieved %s idicator data with %d columns and %d rows"
      % (__idx_type__.upper(),len(clsIndx.data.columns),clsIndx.data.count()))

Wait a moment, retrieving data ...
23/02/08 14:52:49 WARN Utils: Your hostname, FarmRaiderTester resolves to a loopback address: 127.0.1.1; using 192.168.124.15 instead (on interface enp2s0)
23/02/08 14:52:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/08 14:52:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/08 14:52:50 WARN FileSystem: Cannot load filesystem: java.util.ServiceConfigurationError: org.apache.hadoop.fs.FileSystem: com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem Unable to get public no-arg constructor
23/02/08 14:52:50 WARN FileSystem: java.lang.NoClassDefFoundError: com/google/api/client/auth/oauth2/Credential
23/02/08 14:52:50 WARN FileSystem: java.lang.ClassNotFoundException: com.google.api.client.auth.oauth2.Credential


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/08 14:52:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


[Error]function <index_wrapper> must be real number, not NoneType


Retrieved DMI idicator data with 18 columns and 24 rows


### ADX
* Non-directional and ranges from a low of 0-100. 
* Registers trend strength whether price is trending up or down
* Plotted in the same window as the DMI lines, from which ADX is derived 

In [4]:
__idx_type__='adx'
_kwargs={
    "LOGCOLNAME" :'log_ror',
    "PARTCOLNAME":"asset_name",
    "VALCOLNAME" :"mcap_value",
    "PREVALCOLNAME":'mcap_prev_val',
    "DIFFCOLNAME":'mcap_diff',
    "WINLENGTH":7,
    "WINUNIT":'DAY',
}
_idx = clsIndx.get_index(
    portfolio=_mpt_port,
    date=_coll_dt,
    index_type=__idx_type__,
    **_kwargs,
)

print("Completed index computation for %s = %0.4f"
      % (__idx_type__.upper(),_idx))

Wait a moment, retrieving data ...
23/02/07 10:36:01 WARN Utils: Your hostname, FarmRaiderTester resolves to a loopback address: 127.0.1.1; using 192.168.124.15 instead (on interface enp2s0)
23/02/07 10:36:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/07 10:36:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/07 10:36:02 WARN FileSystem: Cannot load filesystem: java.util.ServiceConfigurationError: org.apache.hadoop.fs.FileSystem: com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem Unable to get public no-arg constructor
23/02/07 10:36:02 WARN FileSystem: java.lang.NoClassDefFoundError: com/google/api/client/auth/oauth2/Credential
23/02/07 10:36:02 WARN FileSystem: java.lang.ClassNotFoundException: com.google.api.client.auth.oauth2.Credential


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<class 'pyspark.sql.dataframe.DataFrame'> 0.4203415087858535
Completed index computation for ADX = 0.4203


In [12]:
clsIndx.data.select(
    F.col('mcap_date'),
#     F.col('asset_name'),
#     F.col('+DI'),
#     F.col('-DI'),
#     F.col('weight'),
    F.col('weighted_+DI'),
    F.col('weighted_-DI'),
    F.col('ADX')
).filter(F.col("asset_name")=='matryx').show()

+-------------------+--------------------+--------------------+--------------------+
|          mcap_date|        weighted_+DI|        weighted_-DI|                 ADX|
+-------------------+--------------------+--------------------+--------------------+
|2022-01-03 00:00:00|                null|                null|                null|
|2022-01-04 00:00:00|                null| 0.34821271988748376|                null|
|2022-01-05 00:00:00| 0.17410635994374188| 0.06964254397749675|-0.42857142857142855|
|2022-01-06 00:00:00| 0.06700901117698771| 0.11607079055825462| 0.26798029556650244|
|2022-01-07 00:00:00|0.055675035357370006| 0.08705317997187094| 0.21984542118822697|
|2022-01-08 00:00:00| 0.06964254397749675|  0.0693047776392059|-0.00243089492018...|
|2022-01-09 00:00:00|0.058035569385487255|0.057719044023109525|-0.00273445138001...|
|2022-01-10 00:00:00|0.046822423379670505| 0.04974462452496627| 0.03026085200597144|
+-------------------+--------------------+--------------------+--

In [11]:
clsIndx.data.printSchema()

root
 |-- mcap_date: timestamp (nullable = true)
 |-- asset_name: string (nullable = true)
 |-- mcap_value: decimal(38,16) (nullable = true)
 |-- mcap_value_prev_val: decimal(38,16) (nullable = true)
 |-- mcap_diff: decimal(38,15) (nullable = true)
 |-- log_ror: double (nullable = true)
 |-- +DM: decimal(38,18) (nullable = true)
 |-- -DM: decimal(38,18) (nullable = true)
 |-- sm_sum_+DM: decimal(38,18) (nullable = true)
 |-- sm_avg_+DM: decimal(38,22) (nullable = true)
 |-- sm_sum_-DM: decimal(38,18) (nullable = true)
 |-- sm_avg_-DM: decimal(38,22) (nullable = true)
 |-- shift_+DM: decimal(38,18) (nullable = true)
 |-- shift_-DM: decimal(38,18) (nullable = true)
 |-- smooth_+DM: decimal(38,16) (nullable = true)
 |-- smooth_-DM: decimal(38,16) (nullable = true)
 |-- +DI: decimal(38,6) (nullable = true)
 |-- -DI: decimal(38,6) (nullable = true)
 |-- weight: double (nullable = true)
 |-- weighted_+DI: double (nullable = true)
 |-- weighted_-DI: double (nullable = true)
 |-- ADX: double (

In [25]:
adx_df = clsIndx.data.groupBy('mcap_date')\
                .avg('weighted_+DI','weighted_-DI','ADX').toPandas()
adx_df.sort_values(by='mcap_date',inplace=True)
adx_df

,mcap_date,avg(weighted_+DI),avg(weighted_-DI),avg(ADX)
0,2022-01-03,NaN,NaN,NaN
4,2022-01-04,NaN,0.333333,NaN
3,2022-01-05,0.138670,0.066822,-0.428571
6,2022-01-06,0.093739,0.110546,0.094980
7,2022-01-07,0.042850,0.083333,0.326906
2,2022-01-08,0.066667,0.064647,-0.025624
1,2022-01-09,0.054071,0.055450,0.022160
5,2022-01-10,0.046645,0.047462,0.007406


In [26]:
import plotly.express as px
from pyspark.sql import functions as F

# adx_df = clsIndx.data.filter(F.col("asset_name")=="charg_coin").toPandas()
adx_df = clsIndx.data.groupBy('mcap_date')\
                .avg('weighted_+DI','weighted_-DI','ADX').toPandas()
adx_df.sort_values(by='mcap_date',inplace=True)
_min_date = adx_df["mcap_date"].min()
_max_date = adx_df["mcap_date"].max()
_title = "Weighted ADX "+str(_min_date)+" to "+str(_max_date)
fig = px.line(adx_df, x="mcap_date",
              y=['avg(weighted_+DI)','avg(weighted_-DI)','avg(ADX)'],
#               color='red',
#               hover_data={"mcap_date": "|%B %d, %Y", "asset_name": True,},
              title=_title,)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()


# SMA

In [260]:
_mcap_log_ror=clsIndx.data.na.drop(subset=['log_ror'])

In [264]:
kwargs={
    "DATETIMEATTR":'mcap_date',
    "WINLENGTH":7,
    "WINUNIT":'DAY',
}
_sma_sdf=clsStat.simple_moving_stats(
    column='log_ror',   # column name to apply the rolling computation
    stat_op="avg", # stat operation sum, mean or standard deviation
    data=_mcap_log_ror,   # data set
    **kwargs,    # 
)


In [266]:
_sma_sdf.select(["log_ror","rolling_avg_log_ror"]).show()

23/02/01 16:09:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/02/01 16:09:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 16:09:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/01 1